In [3]:
# Get pandas and postgres to work together
import psycopg2 as pg
import pandas as pd
import numpy as np
import pandas.io.sql as pd_sql
import pickle

# We are also going to do some basic viz
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# There is a bug in matplotlib. You cannot set the rc parameters in the same
# cell that you use the "%matplotlib inline" magic command
plt.style.use('ggplot')
plt.rc('font', size=18) 

In [6]:
# Postgres info to connect

connection_args = {
    'host': '54.153.60.68', # You'll have to update this to your IP
    'user': 'ubuntu',    # username
    'dbname': 'airbnb',   # DB that we are connecting to
    'port': 5432         # port we opened on AWS
}

# We will talk about this magic Python trick!
connection = pg.connect(**connection_args)

In [ ]:
cursor = connection.cursor()

In [ ]:
query = """
CREATE TABLE trainuser (
  id VARCHAR(10) PRIMARY KEY,
  date_account_created TEXT,
  timestamp_first_active NUMERIC,
  date_first_booking TEXT,
  gender TEXT,
  age DOUBLE PRECISION,
  signup_method TEXT,
  signup_flow INT,
  language TEXT,
  affiliate_channel TEXT,
  affiliate_provider TEXT,
  first_affiliate_tracked TEXT,
  signup_app TEXT,
  first_device_type TEXT,
  first_browser TEXT,
  country_destination TEXT
);
"""
cursor.execute(query)

In [ ]:
#cursor.execute('commit;')
#cursor.execute("rollback;")

In [ ]:
query = """
CREATE TABLE session (
  user_id VARCHAR(10),
  action TEXT,
  action_type TEXT,
  action_detail TEXT,
  device_type TEXT,
  secs_elapsed REAL
);
"""
cursor.execute(query)

In [ ]:
query = """
CREATE TABLE country (
  country_destination VARCHAR(2) PRIMARY KEY,
  lat_destination REAL,
  lng_destination REAL,
  distance_km REAL,
  destination_km2 REAL,
  destination_language VARCHAR(3),
  language_levenshtein_distance REAL
);
"""
cursor.execute(query)

In [ ]:
query = """
CREATE TABLE population (
  age_bucket TEXT,
  country_destination VARCHAR(2) REFERENCES country,
  gender TEXT,
  population_in_thousands REAL,
  year REAL
);
"""
cursor.execute(query)

In [ ]:
"""
To copy the data into the database
scp user_subset.csv myaws:.

Log on the database and run command
COPY session FROM '/home/ubuntu/session_subset.csv' DELIMITER ',' CSV HEADER;
"""

## Data cleaning

In [ ]:
def get_table_info(cursor, table):
    # Check column data types
    query = """
    SELECT column_name, data_type 
    FROM information_schema.columns
    WHERE table_name = '{}'
    """.format(table)

    cursor.execute(query)
    return cursor.fetchall()

## Convert the date_account_created to date

In [ ]:
# Convert date_account_created from text to timestamp, for further processsing
query = """
ALTER TABLE trainuser ALTER COLUMN date_account_created TYPE TIMESTAMP
using to_date(date_account_created, 'YYYY-MM-DD');
"""

cursor.execute(query)

In [ ]:
# Convert the timestamp into continous variable
query = """
ALTER TABLE trainuser ADD COLUMN account_age INT
"""

cursor.execute(query)

In [ ]:
query = """
UPDATE trainuser 
SET account_age = cast(12 * (date_part('year', date_account_created) - 1970) + date_part('month', date_account_created) AS int8)
"""

cursor.execute(query)

## Convert timestamp_first_active

In [ ]:
## Convert date_account_created from text to timestamp, for further processsing
query = """
ALTER TABLE trainuser ALTER COLUMN timestamp_first_active TYPE TIMESTAMP
using to_date(cast(timestamp_first_active as text), 'YYYYMMDDHH24MISS');
"""

cursor.execute(query)

In [ ]:
# Convert the timestamp into continous variable
query = """
ALTER TABLE trainuser ADD COLUMN active_age INT
"""

cursor.execute(query)

In [ ]:
query = """
UPDATE trainuser 
SET active_age = cast(12 * (date_part('year', timestamp_first_active) - 1970) + date_part('month', timestamp_first_active) AS int8)
"""

cursor.execute(query)

## Convert date_first_booking

In [ ]:
# Seems Postgresql ignores none value pretty smartly
query = """
ALTER TABLE trainuser ALTER COLUMN date_first_booking TYPE TIMESTAMP
using to_date(date_first_booking, 'YYYY-MM-DD');
"""

cursor.execute(query);

In [ ]:
# Convert the timestamp into continous variable
query = """
ALTER TABLE trainuser ADD COLUMN booking_age INT
"""

cursor.execute(query)

In [ ]:
# Convert the timestamp into continous variable
query = """
UPDATE trainuser
SET booking_age = CASE
    WHEN date_first_booking IS NULL THEN 0
    ELSE cast(12 * (date_part('year', date_first_booking) - 1970) + date_part('month', date_first_booking) AS int8)
    END;
"""

cursor.execute(query)

In [ ]:
query = """
SELECT *
FROM trainuser
LIMIT 5;
"""

cursor.execute(query)
pd.DataFrame(cursor.fetchall())

In [ ]:
cursor.execute("rollback")

## Check gender column

In [ ]:
query = """
SELECT gender, count(*)
FROM trainuser
GROUP BY gender
"""

cursor.execute(query)
cursor.fetchall()

# It is clear that gender needs further data cleaning

## Check age range

In [ ]:
query = """
SELECT cast(age as int8) as age_int, count(*)
FROM trainuser
GROUP BY cast(age as int8)
ORDER BY cast(age as int8) DESC
"""

cursor.execute(query)
cursor.fetchall()

# A lot of wrong values and missing values

## Check signup method

In [ ]:
def check_col_hist(cursor, table, column):
    query = """
    SELECT {}, count(*)
    FROM {}
    GROUP BY {}
    ORDER BY count DESC
    """.format(column, table, column)

    cursor.execute(query)
    return pd.DataFrame(cursor.fetchall())

In [ ]:
check_col_hist(cursor, 'trainuser', 'signup_method')

## Check signup flow

In [ ]:
check_col_hist(cursor, 'trainuser', 'signup_flow')

# What do these signup flow numbers mean? Index of signup pages - treat it as categorical variable

## Check language

In [ ]:
check_col_hist(cursor, 'trainuser','language')

In [ ]:
check_col_hist(cursor, 'trainuser','affiliate_channel')

#sem: search engine marketing
#seo: SEO is a marketing discipline focused on growing visibility in organic (non-paid) search engine results.


In [ ]:
check_col_hist(cursor, 'trainuser','affiliate_provider')

In [ ]:
check_col_hist(cursor, 'trainuser','first_affiliate_tracked')

In [ ]:
check_col_hist(cursor, 'trainuser', 'signup_app')

In [ ]:
check_col_hist(cursor, 'trainuser', 'first_device_type')

In [ ]:
check_col_hist(cursor, 'trainuser', 'first_browser')

In [ ]:
check_col_hist(cursor, 'trainuser', 'country_destination')

## Add splitseed to trainuser data

In [ ]:
# Insert a new column
query = """
ALTER TABLE trainuser
ADD COLUMN splitseed REAL;
"""

cursor.execute(query)

In [ ]:
# Insert data into new column
query = """
UPDATE trainuser
SET splitseed=random();
"""

cursor.execute(query)

In [ ]:
query = """
SELECT COUNT(*)
FROM trainuser
WHERE splitseed>.7;
"""

cursor.execute(query)
cursor.fetchone()

In [ ]:
query = """
SELECT COUNT(*)
FROM trainuser
WHERE splitseed<=.7;
"""

cursor.execute(query)
cursor.fetchone()

## Check session data

In [ ]:
# CREATE VIEW
query = """
DROP VIEW session_stats;
"""

cursor.execute(query)

In [ ]:
# CREATE VIEW
query = """
CREATE VIEW session_stats AS
SELECT user_id,  avg(secs_elapsed) AS mean_time, count(*) AS session_counts
FROM session
GROUP BY user_id;
"""

cursor.execute(query)

In [ ]:
query = """
SELECT *
FROM session_stats
ORDER BY session_counts DESC
LIMIT 5;
"""

cursor.execute(query)
pd.DataFrame(cursor.fetchall())

In [ ]:
get_table_info(cursor, 'session')

In [ ]:
check_col_hist(cursor, 'session', 'action')

In [ ]:
check_col_hist(cursor, 'session', 'action_type')

In [ ]:
check_col_hist(cursor, 'session', 'action_detail')

In [ ]:
check_col_hist(cursor, 'session', 'device_type')

In [ ]:
check_col_hist(cursor, 'session', 'secs_elapsed')

In [ ]:
query = """
SELECT count(*)
FROM session;
"""

cursor.execute(query)
cursor.fetchone()

In [ ]:
query = """
SELECT count(*)
FROM trainuser;
"""

cursor.execute(query)
cursor.fetchone()

In [ ]:
query = """
SELECT count(*)
FROM session LEFT JOIN trainuser
ON (session.user_id = trainuser.id)
WHERE trainuser.splitseed <= 0.1
;
"""

cursor.execute(query)
cursor.fetchone()

In [ ]:
cursor.execute("rollback")

In [ ]:
## Join two tables together

query = """
CREATE VIEW session_booking AS
SELECT session.*, trainuser.*
FROM session LEFT JOIN trainuser 
ON (session.user_id = trainuser.id);
"""

cursor.execute(query)

In [ ]:
## Join two tables together

query = """
SELECT *
FROM session_booking
WHERE splitseed <= 0.7;
"""



cursor.execute(query)
data = cursor.fetchall()

user_cols = [x[0] for x in get_table_info(cursor, 'trainuser')]
session_cols = [x[0] for x in get_table_info(cursor, 'session')]

train_session = pd.DataFrame(data, columns = user_cols+session_cols)


In [ ]:
query = """
SELECT *
FROM session_booking_train
LIMIT 5
"""

cursor.execute(query)
pd.DataFrame(cursor.fetchall())

## Try to export booking data into python to have a look

In [ ]:
query = """
SELECT *
FROM trainuser
WHERE splitseed <= 0.7;
"""
cursor.execute(query)
train_data = cursor.fetchall()

user_cols = [x[0] for x in get_table_info(cursor, 'trainuser')]

train_user = pd.DataFrame(train_data, columns = user_cols)

In [ ]:
train_user.head()

In [ ]:
y_train = pd.DataFrame(train_user['country_destination'].values, columns=['country'], index=train_user['id'])
X_train = train_user.drop(('country_destination'), axis=1)

In [ ]:
y_train.head()

In [ ]:
with open('./data/processed/X_train.pkl', 'wb') as picklefile:
    pickle.dump(X_train, picklefile)

In [ ]:
with open('./data/processed/y_train.pkl', 'wb') as picklefile:
    pickle.dump(y_train, picklefile)

In [ ]:
query = """
SELECT *
FROM trainuser
WHERE splitseed > 0.7;
"""
cursor.execute(query)
test_data = cursor.fetchall()
user_cols = [x[0] for x in get_table_info(cursor, 'trainuser')]
test_user = pd.DataFrame(test_data, columns = user_cols)

y_test = test_user['country_destination'][:,np.newaxis]
X_test = test_user.drop(('country_destination'), axis=1)

with open('./data/processed/X_test.pkl', 'wb') as picklefile:
    pickle.dump(X_test, picklefile)
    
with open('./data/processed/y_test.pkl', 'wb') as picklefile:
    pickle.dump(y_test, picklefile)

### Deal with session data

In [ ]:
query = """
SELECT user_id, action, action_type, action_detail, device_type, secs_elapsed
FROM session_booking
WHERE splitseed <=0.7;
"""

cursor.execute(query)
session_train = cursor.fetchall()

In [ ]:
session_cols = [x[0] for x in get_table_info(cursor, 'session')]

session_train = pd.DataFrame(session_train, columns=session_cols)

In [ ]:
with open('./data/raw/session_train.pkl', 'wb') as picklefile:
    pickle.dump(session_train, picklefile)

In [ ]:
query = """
SELECT user_id, action, action_type, action_detail, device_type, secs_elapsed
FROM session_booking
WHERE splitseed > 0.7;
"""

cursor.execute(query)
session_test = cursor.fetchall()
session_test = pd.DataFrame(session_test, columns=session_cols)

with open('./data/raw/session_test.pkl', 'wb') as picklefile:
    pickle.dump(session_test, picklefile)

In [ ]:
query = """
SELECT *
FROM trainuser
LIMIT 5
"""

cursor.execute(query)
pd.DataFrame(cursor.fetchall())

In [ ]:
get_table_info(cursor, 'session')

In [ ]:
# Convert the timestamp into continous variable
query = """
ALTER TABLE session ADD COLUMN session_under_2 INT
"""
cursor.execute(query)

In [ ]:
query = """
UPDATE session
SET session_under_2 = 
CASE WHEN secs_elapsed <= 120 THEN 1
ELSE 0 END;
"""
cursor.execute(query)

In [ ]:
# Convert the timestamp into continous variable
query = """
ALTER TABLE session ADD COLUMN session_between_2_and_5 INT
"""
cursor.execute(query)

In [ ]:
query = """
UPDATE session
SET session_between_2_and_5 = 
CASE WHEN (secs_elapsed >= 120 AND secs_elapsed <= 300) THEN 1
ELSE 0 END;
"""
cursor.execute(query)

In [ ]:
query = """
SELECT secs_elapsed
FROM session;
"""

cursor.execute(query)
mins = cursor.fetchall()